In [4]:
%load_ext autoreload
%autoreload 2

In [101]:
# Scientific Computing
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats

import bottleneck as bn

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from tools.tools import *
from tools.io import *
from tools.methods import *

sns.set(context='talk', style='whitegrid', font_scale=1.3)

In [ ]:
fu.ev(dat)

In [6]:
def sensitivity_length(filt, fu, fv, fzeta):
    '''
    least square method varying length of hexagons
    '''
    N = 2500
    M = 3
    llist = np.arange(1, 30, 1).astype(float)
    llist = np.insert(llist, 0, 0.5)
    skew=1

    error_l = []
    error_l_ci = []
    for l, L in enumerate(llist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = fu.ev(xi, yi)
        vi = fu.ev(xi, yi)
        zeta_at_mean = fzeta.ev(bn.nanmean(xi, axis=1), bn.nanmean(yi, axis=1))

        vort_drifters = np.zeros(N)
        for i in range(N):
            vort_drifters[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'solve')
        
        # compute R2 and confidence interval
        error_l.append(stats.pearsonr(zeta_at_mean, vort_drifters)[0]**2)
        error_l_ci.append(bootstrap_ci(zeta_at_mean, vort_drifters, N))

    # save in dataframe
    df = pd.DataFrame(index=np.asarray(llist))
    df['error'] = np.asarray(error_l)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

def sensitivity_aspect(filt, fu, fv, fzeta):
    '''
    least square method varying number of drifter per cluster
    '''
    N = 2500
    L = 2
    M = 3
    skewlist = np.arange(1, 30, 1)

    aspect = []
    error_l = []
    error_l_ci = []
    for l, skew in enumerate(skewlist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = fu.ev(xi, yi)
        vi = fu.ev(xi, yi)
        zeta_at_mean = fzeta.ev(bn.nanmean(xi, axis=1), bn.nanmean(yi, axis=1))

        vort_drifters = np.zeros(N)
        for i in range(N):
            vort_drifters[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'lstsq')

        # compute aspect ratio, R2 and confidence interval
        aspect.append(calc_aspect(xi[0, :], yi[0, :]))
        error_l.append(stats.pearsonr(zeta_at_mean, vort_drifters)[0]**2)
        error_l_ci.append(bootstrap_ci(zeta_at_mean, vort_drifters, N))
    
    # save in dataframe
    df = pd.DataFrame(index=np.asarray(aspect))
    df['error'] = np.asarray(error_l)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

def sensitivity_number(filt, fu, fv, fzeta):
    '''
    least square method varying number of drifter per cluster
    '''
    N = 2500
    L = 10
    mlist = np.arange(3, 21)
    skew=1

    error_l = []
    error_l_ci = []
    for l, M in enumerate(mlist):
        # make polygons
        xi, yi = make_n_hexs(L, skew, N, M)

        # find the u,v at the polygon vertices
        ui = np.zeros((N, M))
        vi = np.zeros((N, M))
        zeta_at_mean = np.zeros(N)
        for i in range(N):
            for j in range(xi[1, :].size):
                ui[i, j] = fu.ev(yi[i, j], xi[i, j])
                vi[i, j] = fv.ev(yi[i, j], xi[i, j])
            # find zeta at the polygon center
            zeta_at_mean[i] = fzeta.ev(bn.nanmean(yi[i]), bn.nanmean(xi[i]))

        # estimate vorticity from the velocities
        vort_drifters = np.zeros(N)
        for i in range(N):
            vort_drifters[i], _, _ = least_square_method(
                xi[i, :], yi[i, :], ui[i, :], vi[i, :], 'inv')
        
        # compute R2 and confidence interval
        error_l.append(stats.pearsonr(zeta_at_mean, vort_drifters)[0]**2)
        error_l_ci.append(bootstrap_ci(zeta_at_mean, vort_drifters, N))

    # save in dataframe
    df = pd.DataFrame(index=np.asarray(mlist))
    df['error'] = np.asarray(error_l)
    df['ci_low'] = np.asarray(error_l_ci)[:, 0]
    df['ci_high'] = np.asarray(error_l_ci)[:, 1]
    df['filter'] = filt

    return df

In [62]:
# %% MAIN
data_path = '../../data/'
zgrid_path = data_path+'psom/zgrid.out'
model_path = data_path+'psom/full_08325.cdf'

# dat = read_model_field(snakemake.input[0], snakemake.input[1])
dat = read_model_field(zgrid_path, model_path) 
dat = dat.rename({'xc':'x'})
dat = dat.rename({'yc':'y'})
dat = dat.set_coords({'x','y'})
dat = dat.transpose('x','y')


fu, fv, fzeta = filter_fields(dat)

In [41]:
length_bucket = []
number_bucket = []
aspect_bucket = []
for i, filt in enumerate([0, 10, 20]):
    print(filt)
    length_bucket.append(
        sensitivity_length(filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))
    number_bucket.append(
        sensitivity_number(filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))
    aspect_bucket.append(
        sensitivity_aspect(filt, fu=fu[i], fv=fv[i], fzeta=fzeta[i]))

0
10
20


In [42]:
pd.concat(length_bucket).reset_index().to_feather(data_path+'psom/sensitivity_length.feather')
pd.concat(number_bucket).reset_index().to_feather(data_path+'psom/sensitivity_number.feather')
pd.concat(aspect_bucket).reset_index().to_feather(data_path+'psom/sensitivity_aspect.feather')